In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=ff07ad6d069453dd980b6fa98345b81602858765323855f484d78dfe0f65b489
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
appName='lab11'
master="local[*]"
conf = SparkConf().setAppName(appName).setMaster(master)


In [ ]:
import pandas as pd
import pyspark.sql as ps
from pyspark.sql.functions import length, col, collect_list, substring

In [ ]:
df=spark.read.csv('Movies.csv', header=True)

In [ ]:
df.show()

+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|Year|Length|               Title|  Genre|               Actor|          Actress|            Director|Popularity|Awards|              Image|
+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|1990|   111|Tie Me Up! Tie Me...| Comedy|     BanderasAntonio|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1991|   113|          High Heels| Comedy|          BoséMiguel|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1983|   104|        Dead ZoneThe| Horror|   WalkenChristopher|      AdamsBrooke|     CronenbergDavid|        79|    No|   NicholasCage.png|
|1979|   122|                Cuba| Action|         ConnerySean|      AdamsBrooke|       LesterRichard|         6|    No|    seanConnery.png|
|1978|    94|

In [ ]:
#Q1
df.filter(df.Awards=='Yes').filter(df.Genre=='Action').select("Title","Year","Director").show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



In [ ]:
#Q2
df.filter(df.Awards=='Yes').select("Actor","Title","Director").show()

+--------------------+--------------------+-------------------+
|               Actor|               Title|           Director|
+--------------------+--------------------+-------------------+
|       AhlstedtBörje| Fanny and Alexander|      BergmanIngmar|
|TrintignantJean-L...|     A Man & a Woman|      LelouchClaude|
|TrintignantJean-L...|Un Hombre y una M...|      LelouchClaude|
|       AlterioHector|   Official StoryThe|         PuenzoLuiz|
|      SjöströmVictor|   Wild Strawberries|      BergmanIngmar|
|   BjörnstrandGunnar|Through a Glass D...|      BergmanIngmar|
|     JosephsonErland|    Cries & Whispers|      BergmanIngmar|
|  PlummerChristopher|   Sound of MusicThe|         WiseRobert|
| LaFontJean-Philippe|     Babette's Feast|        AxelGabriel|
|        BalsamMartin|Murder on the Ori...|        LumetSidney|
|      HopkinsAnthony|     Elephant ManThe|         LynchDavid|
|          WallaceDee|E. T. The Extra-T...|    SpielbergSteven|
|           CaanJames|              Mise

In [ ]:
#Q3
df.filter(df.Awards=='No').sort(df.Popularity.desc()).select("Title").limit(10).show()

+--------------------+
|               Title|
+--------------------+
|        Tango & Cash|
|              Plenty|
|Masque of the Red...|
|           Lionheart|
|        Judge Priest|
|           Dead-Bang|
|       Basil The Rat|
|               Glitz|
|            Morituri|
|Naked Gun: From t...|
+--------------------+



In [ ]:
#Q4
df.filter(df.Year<=1980).sort(df.Popularity.asc()).select("Title").limit(10).show()

+------------------+
|             Title|
+------------------+
|   White Lightning|
|      Desert Rider|
|      Drop KickThe|
| Bank on the Stars|
|Shout at the Devil|
|      It's My Turn|
|           Shalako|
|     Anna Christie|
|           Airport|
| Tales of Tomorrow|
+------------------+



In [ ]:
#Q5
df.groupBy(df.Genre).agg(ps.functions.avg(df.Length).alias('Average Length of Movies')).show()

+---------------+------------------------+
|          Genre|Average Length of Movies|
+---------------+------------------------+
|          Crime|                    66.0|
|        Romance|                   127.0|
|      Adventure|                   119.0|
|           null|                   120.5|
|          Drama|      113.30455259026688|
|            War|               116.90625|
|        Fantasy|                   102.0|
|        Mystery|      103.00990099009901|
|          Music|      100.48780487804878|
|Science Fiction|      106.47368421052632|
|         Horror|       93.92727272727272|
|          Short|                    40.0|
|        Western|        93.0091743119266|
|         Comedy|       96.50540540540541|
|         Action|                   104.5|
|       Westerns|                   124.8|
+---------------+------------------------+



In [ ]:
from pyspark.sql.functions import count,when

In [ ]:
#Q6
df.filter(col("Genre") == "Comedy") \
  .groupBy("Actor", "Actress") \
  .agg(count("*").alias("count")) \
  .filter(col("count") > 3) \
  .filter(col("Actress")!='null')\
  .show()

+------------+----------------+-----+
|       Actor|         Actress|count|
+------------+----------------+-----+
|TracySpencer|HepburnKatharine|    6|
|  AllenWoody|     KeatonDiane|    5|
+------------+----------------+-----+



In [ ]:
#Q7
df.where((col('Genre') == 'Comedy') & (col('Genre') == 'Drama')) \
  .select(col('Actor')) \
  .show()

+-----+
|Actor|
+-----+
+-----+



In [ ]:
#Q8
df.where((col('Genre') == 'Comedy') | (col('Genre') == 'Drama')) \
  .select(col('Actor')) \
  .show()

+--------------------+
|               Actor|
+--------------------+
|     BanderasAntonio|
|          BoséMiguel|
|         GereRichard|
|     BergenRobert D.|
|  LambertChristopher|
|     DepardieuGérard|
|       AhlstedtBörje|
|         TognazziUgo|
|TrintignantJean-L...|
|TrintignantJean-L...|
|TrintignantJean-L...|
|       AlterioHector|
|      HopkinsAnthony|
|           WayneJohn|
|         BridgesJeff|
|          MurrayBill|
|            PennSean|
|         CageNicolas|
|       AndersonKevin|
|          NewmanPaul|
+--------------------+
only showing top 20 rows



In [ ]:
#Q9
df.select(df.Actor) \
  .where(df.Genre!='Comedy').show()

+--------------------+
|               Actor|
+--------------------+
|   WalkenChristopher|
|         ConnerySean|
|         GereRichard|
|          MooreRoger|
|        ConnorsChuck|
|     BergenRobert D.|
|  LambertChristopher|
|     DepardieuGérard|
|       AhlstedtBörje|
|         TognazziUgo|
|TrintignantJean-L...|
|TrintignantJean-L...|
|TrintignantJean-L...|
|       AlterioHector|
|      HopkinsAnthony|
|         LongdenJohn|
|         BridgesJeff|
|        FordHarrison|
|SchwarzeneggerArnold|
|         GloverDanny|
+--------------------+
only showing top 20 rows



In [ ]:
#Q10
from pyspark.sql.functions import min,max,avg
df.groupBy("Actor") \
  .agg(min("Popularity").alias("min"), \
  max("Popularity").alias("max"), \
  avg("Popularity").alias("avg")) \
  .filter(df.Actor!='null') \
  .show()

+------------------+---+---+-----------------+
|             Actor|min|max|              avg|
+------------------+---+---+-----------------+
|        AbelAlfred| 49| 49|             49.0|
|  AbrahamF. Murray|  6|  6|              6.0|
|    AdolphsonEdvin| 26| 72|             49.0|
|       AherneBrian| 57| 57|             57.0|
|     AhlstedtBörje| 81| 81|             81.0|
|       AielloDanny| 20|  5|             12.5|
|         AkanTarik| 53| 53|             53.0|
|    AlbaicínRafael| 55| 55|             55.0|
|      AlbertEdward| 82| 82|             82.0|
|          AldaAlan| 12| 53|             32.5|
|         AllenBill| 75| 75|             75.0|
|        AllenWoody| 12| 84|51.27272727272727|
|     AlterioHector| 39| 39|             39.0|
|         AmecheDon| 45| 45|             45.0|
|     AndersonKevin| 53| 53|             53.0|
|   AnderssonWiktor| 66| 66|             66.0|
|AngladeJean-Hughes| 71| 71|             71.0|
|       AnneseFrank| 45| 45|             45.0|
|        Apfe

In [ ]:
#Q11
from pyspark.sql.functions import floor
df.where(df.Year>=1960) \
  .withColumn("decade", floor(df.Year/10)*10) \
  .groupBy("decade") \
  .agg(count("*").alias("count")) \
  .sort("decade") \
  .show()

+------+-----+
|decade|count|
+------+-----+
|  1960|  154|
|  1970|  244|
|  1980|  607|
|  1990|  345|
+------+-----+



In [ ]:
#Q12
df.groupBy("Year")\
  .agg(count("*").alias('count'))\
  .show()

+----+-----+
|Year|count|
+----+-----+
|1953|   16|
|1957|   16|
|1987|  102|
|1956|   15|
|1936|    6|
|1958|   12|
|1943|    8|
|1972|   24|
|1931|    9|
|1988|   96|
|1926|    4|
|1938|    8|
|1932|    8|
|1977|   29|
|1971|   23|
|1984|   38|
|1982|   37|
|1941|    7|
|1965|   12|
|1962|   15|
+----+-----+
only showing top 20 rows



In [ ]:
#Q13
df.filter(df.Length>100)\
  .groupBy("Year","Genre")\
  .agg(count("*").alias('count'))\
  .show()

+----+---------------+-----+
|Year|          Genre|count|
+----+---------------+-----+
|1984|         Comedy|    4|
|1967|          Drama|    3|
|1958|          Drama|    3|
|1968|          Music|    1|
|1990|        Mystery|    4|
|1992|      Adventure|    1|
|1933|          Drama|    1|
|1982|Science Fiction|    1|
|1978|Science Fiction|    1|
|1951|            War|    1|
|1964|       Westerns|    1|
|1988|         Action|    3|
|1975|         Action|    4|
|1992|          Drama|   18|
|1969|        Western|    5|
|1951|        Mystery|    1|
|1962|        Mystery|    1|
|1950|        Mystery|    1|
|1980|        Mystery|    1|
|1990|        Western|    1|
+----+---------------+-----+
only showing top 20 rows



In [ ]:
#Q14
df.where(df.Year<1990)\
  .sort(df.Title)\
  .show()

+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|Year|Length|               Title|          Genre|               Actor|          Actress|           Director|Popularity|Awards|              Image|
+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|1968|   139|2001: A Space Ody...|Science Fiction|          DulleaKeir|   TyzackMargaret|     KubrickStanley|        83|    No|   NicholasCage.png|
|1982|    92|             48 Hrs.|         Action|           NolteNick|   O'TooleAnnette|         HillWalter|        67|    No|   NicholasCage.png|
|1963|   138|               8 1/2|          Drama| MastroianniMarcello| CardinaleClaudia|    FelliniFederico|        80|   Yes|   NicholasCage.png|
|1966|    95|A Big Hand for th...|         Comedy|          FondaHenry|   WoodwardJoanne|        CookFielder|   

In [ ]:
#Q15
df.where(length(df.Title)>50).show()

+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|Year|Length|               Title| Genre|     Actor|       Actress|Director|Popularity|Awards|           Image|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|1979|    90|Fawlty TowersGour...|Comedy|CleeseJohn|ScalesPrunella|    null|        46|    No|NicholasCage.png|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+

